<a href="https://colab.research.google.com/github/silvi3110/sis-252-inteligencia-artificial/blob/main/Introducci%C3%B3n_a_la_Inteligencia_Artificial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ejemplo de Enfoque Simbólico (GOFAI)
# Sistema experto médico básico con reglas lógicas explícitas

def diagnosticar(sintomas):
  if "fiebre" in sintomas and "tos" in sintomas and "dolor_garganta" in sintomas:
    return "Posible gripe"
  elif "fiebre" in sintomas and "dolor_cabeza" in sintomas and "rigidez_cuello" in sintomas:
    return "Posible meningitis"
  elif "estornudos" in sintomas and "congestion" in sintomas:
    return "Posible resfriado común"
  elif "dolor_estomago" in sintomas and "nauseas" in sintomas:
    return "Posible intoxicación alimentaria"
  else:
    return "Diágnostico no determinado"

paciente_1 = ["fiebre", "tos", "dolor_garganta"]
paciente_2 = ["estornudos", "congestion"]
paciente_3 = ["dolor_estomago", "nauseas"]

print("Paciente 1:", diagnosticar(paciente_1))
print("Paciente 2:", diagnosticar(paciente_2))
print("Paciente 3:", diagnosticar(paciente_3))

Paciente 1: Posible gripe
Paciente 2: Posible resfriado común
Paciente 3: Posible intoxicación alimentaria


In [ ]:
# Clasificación con Naive (GaussianNB) en el dataset Breast Cancer
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

nb = GaussianNB()
nb.fit(X_train, y_train)

y_pred_nb = nb.predict (X_test)
print("Naive Bayes - accuracy:", accuracy_score (y_test, y_pred_nb))
print(confusion_matrix (y_test, y_pred_nb))
print(classification_report (y_test, y_pred_nb))

Naive Bayes - accuracy: 0.9370629370629371
[[46  7]
 [ 2 88]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91        53
           1       0.93      0.98      0.95        90

    accuracy                           0.94       143
   macro avg       0.94      0.92      0.93       143
weighted avg       0.94      0.94      0.94       143



In [ ]:
# Clasificación con Red Neuronal (MPL) en el mismo dataset
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

mlp = make_pipeline(
    StandardScaler(),
    MLPClassifier(hidden_layer_sizes=(32, 16), # 2 capas densas pequeñas
                  activation="relu",
                  solver="adam",
                  max_iter=500,
                  random_state=42)
)

mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

print("MPL - accuracy:", accuracy_score(y_test, y_pred_mlp))
print(confusion_matrix(y_test, y_pred_mlp))
print(classification_report(y_test, y_pred_mlp))

MPL - accuracy: 0.965034965034965
[[52  1]
 [ 4 86]]
              precision    recall  f1-score   support

           0       0.93      0.98      0.95        53
           1       0.99      0.96      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.97      0.96       143
weighted avg       0.97      0.97      0.97       143



In [ ]:
# Programación Genética
!pip install deap --quiet

import operator, random
import numpy as np
from deap import base, creator, tools, gp
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1) Dataset real
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# 2) Definición de operadores básicos
pset = gp.PrimitiveSet("MAIN",  X.shape[1])
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(np.sin, 1)
pset.addPrimitive(np.cos, 1)
pset.addEphemeralConstant("rand", lambda: random.randint(-1,1))

# 3) Definición de tipos de individuo y fitness
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 4) Evaluación: usamos accuracy como fitness
def eval_individual(individual):
  func = toolbox.compile(expr=individual)
  preds = [1 if func(*x) > 0 else 0 for x in X_train]
  return accuracy_score(y_train, preds),

toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", eval_individual)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)

# 5) Evolución
pop = toolbox.population(n=50)
for gen in range(10):
  offspring = tools.selTournament(pop, len(pop), tournsize=3)
  offspring = list(map(toolbox.clone, offspring))
  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if random.random() < 0.5:
        toolbox.mate(child1, child2)
  for mutant in offspring:
    if random.random() < 0.2:
        toolbox.mutate(mutant)
  for ind in offspring:
      ind.fitness.values = toolbox.evaluate(ind)
  pop[:] = offspring
  print(f"Generación {gen+1}: Mejor fitness = {max(ind.fitness.values[0] for ind in pop)}")

# 6) Mejor individuo y prueba
best_ind = tools.selBest(pop, 1)[0]
func_best = toolbox.compile(expr=best_ind)
y_pred = [1 if func_best(*x) > 0 else 0 for x in X_test]
print("\nAccury en test:", accuracy_score(y_test, y_pred))
print("\Mejor fórmula encontrada:", best_ind)

/usr/local/lib/python3.11/dist-packages/deap/gp.py:257: RuntimeWarning: Ephemeral rand function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Generación 1: Mejor fitness = 0.6291079812206573
Generación 2: Mejor fitness = 0.6408450704225352
Generación 3: Mejor fitness = 0.6502347417840375
Generación 4: Mejor fitness = 0.6502347417840375
Generación 5: Mejor fitness = 0.8028169014084507
Generación 6: Mejor fitness = 0.8028169014084507
Generación 7: Mejor fitness = 0.8028169014084507
Generación 8: Mejor fitness = 0.812206572769953
Generación 9: Mejor fitness = 0.812206572769953
Generación 10: Mejor fitness = 0.812206572769953

Accury en test: 0.8461538461538461
\Mejor fórmula encontrada: sub(sub(ARG28, sin(ARG26)), ARG15)
